In [ ]:
import os 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

import blackhc.laaos

In [ ]:
from pathlib import Path

# Get the current file's directory
current_working_directory = Path.cwd()

# Get the parent directory
parent_path = current_working_directory.parent

# Get the data-out directory
data_out_path = parent_path.joinpath('data-out')
 

In [ ]:
import glob 

output_files = [] 
pattern = str(data_out_path.joinpath(data_out_path, '*.py'))  
for filename in glob.glob(pattern):
    output_files.append(filename)

len(output_files)

In [ ]:
column_names = ["dataset", "var", "acquisition_method", "trial"]
df = pd.DataFrame(columns = column_names)

for filename in output_files:
    file_split = filename.split('/')[-1].split('_')
    dataset = file_split[0]
    acquisition_method = file_split[-1].split('.')[0]
    trial = int(file_split[-2][5:])
    var = float(file_split[2][3:])
    file_saved = blackhc.laaos.safe_load(filename) 
    single_output=[]
    try:
        single_output.append([min(file_saved['initial_samples']['init_y'])])
    except:
        single_output.append([min(file_saved['initial_samples']['int_y'])])
    if 'iterations' in file_saved:
        for item in file_saved['iterations']:
            single_output.append([item['tol']])
        if single_output[-1][0] <= 1e-3:
            ## Append the results when it stops after the target tolerance met
            while len(single_output)<300:
                single_output.append([item['tol']]) 
        tol_dct = dict(zip(iter(range(1,len(single_output)+1)), single_output))
        single_df = pd.DataFrame({'dataset': dataset, 'var': var, 
                                  'acquisition_method': acquisition_method, 
                                  'trial':trial, **tol_dct})
        df = pd.concat([df, single_df], ignore_index = True)

In [ ]:
othermethods =['gp+ucb', 'gp+ei', 'gp+pi', 'gp+mes', 'gp+kg'] 
# 'gp+pi', 'gp+ei', 'gp+lcb', 
color_pool = ['C'+str(i) for i in range(1+len(othermethods))]

 
fig, axes = plt.subplots(1, 4, figsize=(15, 3.5), sharey=False)

for s, dataset in enumerate(['Branin', 'Ackley']):
    if dataset=='Branin':
        d=2
    else:
        d=5
    for v, var in enumerate([0.01,0.04]):
        for j, method in enumerate(othermethods):
    
            df_inv = df.loc[(df['dataset'] == dataset) & (df['acquisition_method'] == method) & (df['var']==var) 
                             & (df['trial'] <= 5)]
            out_inv = df_inv.dropna(axis=1).drop(columns=['dataset','acquisition_method','trial','var']).to_numpy()
            x = np.arange(0,out_inv.shape[-1])
            out_inv = np.abs(out_inv) ** 2
            y = out_inv.mean(0)
            e = out_inv.std(0) # 2*sigma ~ 95% confidence region 
            axes[v+s*2].plot(x, y, '-.', color=color_pool[1+j], label=method)
    
            # axes[v+s*2].fill_between(x, y - e, y + e,
            #                      color=color_pool[1+j], alpha=0.2)
        for ii, method in enumerate(['infmax']):  
            df_inv = df.loc[(df['dataset'] == dataset) & (df['acquisition_method'] == method) & (df['var']==var) 
                             & (df['trial'] <= 5)]
            out_inv = df_inv.dropna(axis=1).drop(columns=['dataset','acquisition_method','trial','var']).to_numpy()
            out_inv = np.abs(out_inv) ** 2
            x = np.arange(0,out_inv.shape[-1])
            y = out_inv.mean(0)
            e = out_inv.std(0) # 2*sigma ~ 95% confidence region 
            axes[v+s*2].plot(x, y, '--', 
                         color=color_pool[ii], 
                         label='infmax')
    
            # axes[v+s*2].fill_between(x, y - e, y + e, 
            #                      color=color_pool[ii], alpha=0.2)
        if dataset == "Branin":
            xlim_upper = 150
            xlim_lower = -1
            ylim_upper = 0.14 
            ylim_lower = -0.02
            
        else:
            xlim_upper = 100
            xlim_lower = -1
            ylim_upper = 90 
            ylim_lower = -5 
            
        axes[v+s*2].hlines(0.01, xlim_lower, xlim_upper, colors='gray', 
                   linestyles=':', label='0.01', linewidth=1.5)    
    
        axes[v+s*2].set_xlim([xlim_lower, xlim_upper])
        axes[v+s*2].set_ylim([ylim_lower, ylim_upper])
        
        axes[v+s*2].set_title(f'{dataset} (d={d}), var={str(var)}')
        if v==0 and s == 0:
            axes[v+s*2].set_ylabel('Regret')
    
handles, labels = axes[v+s*2].get_legend_handles_labels() 
fig.legend(handles, labels, bbox_to_anchor=(1.0, 0.75))
fig.savefig(os.path.join('./plots/',  'paper_noshade.pdf'), bbox_inches = 'tight' )


In [ ]:

othermethods =['gp+ucb', 'gp+ei', 'gp+pi', 'gp+mes', 'gp+kg'] 
# 'gp+pi', 'gp+ei', 'gp+lcb', 
color_pool = ['C'+str(i) for i in range(1+len(othermethods))]

 
fig, axes = plt.subplots(1, 4, figsize=(15, 3.5), sharey=False)

for s, dataset in enumerate(['Dropwave', 'GoldSteinPrice']):
    if dataset=='Ackley':
        d=5
    else:
        d=2
    for v, var in enumerate([0.01,0.04]):
        for j, method in enumerate(othermethods):
    
            df_inv = df.loc[(df['dataset'] == dataset) & (df['acquisition_method'] == method) & (df['var']==var) 
                             & (df['trial'] <= 5)]
            out_inv = df_inv.dropna(axis=1).drop(columns=['dataset','acquisition_method','trial','var']).to_numpy()
            x = np.arange(0,out_inv.shape[-1])
            out_inv = np.abs(out_inv) ** 2
            y = out_inv.mean(0)
            e = out_inv.std(0) # 2*sigma ~ 95% confidence region
            # plt.errorbar(x, y, e, linestyle='None', marker='^', color=i+1)
            # plt.plot(xdata, ydata, 'or')
            axes[v+s*2].plot(x, y, '-.', color=color_pool[1+j], label=method)
    
            # axes[v+s*2].fill_between(x, y - e, y + e,
            #                      color=color_pool[1+j], alpha=0.2)
        for ii, method in enumerate(['infmax']):  
            df_inv = df.loc[(df['dataset'] == dataset) & (df['acquisition_method'] == method) & (df['var']==var) 
                             & (df['trial'] <= 5)]
            out_inv = df_inv.dropna(axis=1).drop(columns=['dataset','acquisition_method','trial','var']).to_numpy()
            out_inv = np.abs(out_inv) ** 2
            x = np.arange(0,out_inv.shape[-1])
            y = out_inv.mean(0)
            e = out_inv.std(0) # 2*sigma ~ 95% confidence region 
            axes[v+s*2].plot(x, y, '--', 
                         color=color_pool[ii], 
                         label='infmax')
    
            # axes[v+s*2].fill_between(x, y - e, y + e, 
            #                      color=color_pool[ii], alpha=0.2)
        if dataset == "GoldSteinPrice":
            xlim_upper = 150
            xlim_lower = -1
            ylim_upper = 4.2
            ylim_lower = -0.1
            
        else:
            xlim_upper = 300
            xlim_lower = -1
            ylim_upper = 0.52
            ylim_lower = -0.02
            
        axes[v+s*2].hlines(0.01, xlim_lower, xlim_upper, colors='gray', 
                   linestyles=':', label='0.01', linewidth=1.5)    
    
        axes[v+s*2].set_xlim([xlim_lower, xlim_upper])
        axes[v+s*2].set_ylim([ylim_lower, ylim_upper])
        
        axes[v+s*2].set_title(f'{dataset} (d={d}), var={str(var)}')
        if v==0 and s == 0:
            axes[v+s*2].set_ylabel('Regret')
    
handles, labels = axes[v+s*2].get_legend_handles_labels()
fig.legend(handles, labels, bbox_to_anchor=(1.0, 0.75))
fig.savefig(os.path.join('./plots/',  'paper_supp_noshade.pdf'), bbox_inches = 'tight' )
